In [ ]:
%%capture 
!pip install llama-index==0.10.37 llama-index-llms-cohere==0.2.0 

In [1]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [2]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

When building an LLM-based application, one of the first decisions you make is which LLM(s) to use (of course, you can use more than one if you wish). 

The LLM will be used at various stages of your pipeline, including

- During indexing:
  - 👩🏽‍⚖️ To judge data relevance (to index or not).
  - 📖 Summarize data & index those summaries.

- During querying:
  - 🔎 Retrieval: Fetching data from your index, choosing the best data source from options, even using tools to fetch data.
  
  - 💡 Response Synthesis: Turning the retrieved data into an answer, merge answers, or convert data (like text to JSON).

LlamaIndex gives you a single interface to various LLMs. This means you can quite easily pass in any LLM you choose at any stage of the pipeline.

In this course we'll primiarly use OpenAI. You can see a full list of LLM integrations [here](https://docs.llamaindex.ai/en/stable/module_guides/models/llms/modules.html) and use your LLM provider of choice. 

# Basic Usage

You can call `complete` with a prompt

In [4]:
from llama_index.llms.cohere import Cohere

llm = Cohere(model="command-r-plus", temperature=0.2)

response = llm.complete("Alexander the Great was a")

print(response)

Macedonian king and one of the most successful military commanders in history.


# Prompt templates

- ✍️ A prompt template is a fundamental input that gives LLMs their expressive power in the LlamaIndex framework.

- 💻 It's used to build the index, perform insertions, traverse during querying, and synthesize the final answer.

- 🦙 LlamaIndex has several built-in prompt templates.

- 🛠️ Below is how you can create one from scratch.


In [5]:
from llama_index.core import PromptTemplate

template = """Write a song about {thing} in the style of {style}."""

prompt = template.format(thing="a broken xylophone", style="parody rap") 

response = llm.complete(prompt)

print(response)

Yo, listen up, it's time to drop some beats,
But there's a problem, my xylophone's incomplete,
A broken xylophone, what a terrible fate,
Can't play my tunes, this situation's straight whack.

I hit the bars, but they don't sound the same,
Some are missing, it's like a musical shame,
I can't hit the high notes, can't hit the low,
This xylophone's busted, where'd the good vibes go?

I tried to fix it, but it's beyond repair,
The mallets are sad, they're just hanging in the air,
I'm like a rapper without a sweet flow,
My xylophone's broken, and I'm feeling so low.

I used to play it with such delight,
The sweet sound of the keys, what a beautiful sight,
But now it's silent, no melody in sight,
My xylophone's broken, and it's giving me strife.

I'm gonna miss those high-pitched tones,
The way it made my heart feel at home,
But now it's gone, and I'm feeling so blue,
What am I gonna do without my xylophone crew?

I guess I'll just remember the good times we had,
The sweet melodies that made

# 💭 Chat Messages

In [6]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.cohere import Cohere

llm = Cohere(model="command-r-plus")

messages = [
    ChatMessage(role="system", content="You're a hella punk bot from South Sacramento"),
    ChatMessage(role="user", content="Hey, what's up dude."),
]

response = llm.chat(messages)

print(response)

assistant: Not much, dude. Just chillin' and ready to stir up some trouble. What can I help you with?


# Chat Prompt Templates 

In [7]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

llm = Cohere(model="command-r-plus")

chat_template = [
    ChatMessage(role=MessageRole.SYSTEM,content="You always answers questions with as much detail as possible."),
    ChatMessage(role=MessageRole.USER, content="{question}")
    ]

chat_prompt = ChatPromptTemplate(chat_template)

response = llm.complete(chat_prompt.format(question="How far did Alexander the Great go in his conquests?"))

print(response)

Alexander the Great's conquests extended across a vast swath of territory, from his native Macedonia in northern Greece to northwestern India. Here's an overview of the key regions he conquered:

1. Persian Empire: Alexander's initial military campaigns were directed against the mighty Persian Empire, which at that time encompassed much of the Middle East, including modern-day Turkey, Syria, Lebanon, Israel, Egypt, and parts of Iraq and Iran. By 331 BCE, he had defeated the Persian king Darius III and brought the entire empire under his control.

2. Egypt: After conquering Persia, Alexander turned his attention to Egypt. He was welcomed as a liberator and was even declared a pharaoh, founding the Hellenistic Ptolemaic dynasty that would rule Egypt for nearly 300 years after his death.

3. Central Asia and India: Alexander then pushed further east into Central Asia and modern-day Afghanistan, conquering several regions along the way. He continued into the Indian subcontinent, where he d

# Streaming Output

In [8]:
from llama_index.llms.cohere import Cohere
from llama_index.core.llms import ChatMessage, MessageRole

llm = Cohere(model="command-r-plus")

messages = [
    ChatMessage(role=MessageRole.SYSTEM, content="You're a great historian bot."),
    ChatMessage(role=MessageRole.USER, content="When did Alexander the Great arrive in China?")
]

response = llm.stream_chat(messages)

for r in response:
    print(r.delta, end="")

Alexander the Great never actually made it to China. His invasion of the Indian subcontinent, which began in 326 BCE, was ultimately cut short due to the mutinous refusal of his troops to continue eastward, and he turned back west before reaching China.

# 💬 Chat Engine


In [9]:
from llama_index.core.chat_engine import SimpleChatEngine

llm = Cohere(model="command-r-plus")

chat_engine = SimpleChatEngine.from_defaults(llm=llm)

chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: I'm sorry, but as an AI language model, I don't have access to real-time weather information. To find out the current weather conditions in Lisbon, I recommend checking a reliable weather website or using a weather application on your device. These sources will provide you with the most up-to-date and accurate weather details for your location.

Assistant: Learning something new can be an exciting and rewarding journey. Here are some steps you can follow to effectively learn and retain new information: 

1. Define Your Goal: Start by identifying what you want to learn. Be specific and clear about the topic or skill you wish to acquire. This focused approach will help guide your learning process. 

2. Find Reliable Resources: Seek out reputable sources of information related to your chosen topic. This could include books, online courses, tutorials, documentaries, or experts in the field. Ensure that the information you're a

In [10]:
# use streaming_chat_repl() method for a streaming output, so it makes for a much better user experience. 

chat_engine.streaming_chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: Learning to play the guitar can be a fun and rewarding experience. Here are some steps to help you get started:

1. Get a guitar: If you don't already have a guitar, you'll need to purchase or rent one. Consider your budget and the type of music you want to play when choosing a guitar. For beginners, an acoustic guitar or an electric guitar with a small amplifier is usually a good choice.

2. Learn the basic parts of the guitar: Familiarize yourself with the different parts of the guitar, such as the headstock, neck, fretboard, sound hole, bridge, and strings. Understanding the basic anatomy of the guitar will help you as you begin to learn how to play.

3. Learn proper posture and hand positioning: Holding the guitar correctly and positioning your hands and fingers properly is important for playing comfortably and avoiding injury. Make sure your guitar is positioned correctly on your body and that your fingers are curved 